In [1]:
# 导入必要的库
import os
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import random
import csv

In [ ]:
# 检查CUDA是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Data_path = '/home/ubuntu/data/workspace/Li/Data/'
save_path = '/home/ubuntu/data/workspace/Li/Model/Python_code_FFT_Amp/'

In [3]:
M = 9900
batch_size = 1

# 定义数据集类
class SignalDataset(Dataset):
    def __init__(self, generation_signals, labels, experimental_signal):
        self.generation_signals = torch.tensor(generation_signals, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        self.experimental_signal = torch.tensor(experimental_signal, dtype=torch.float32)

    def __len__(self):
        return len(self.generation_signals)

    def __getitem__(self, idx):
        return {
            'generation_signal': self.generation_signals[idx],
            'labels': self.labels[idx],
            'experimental_signal': self.experimental_signal[idx]
        }

In [4]:
class ConvAutoencoder(nn.Module):
    def __init__(self, signal_length, label_dim, output_shape, hidden_dim, dropout_rate):
        super(ConvAutoencoder, self).__init__()
        
        # 输入信号处理路径（编码器部分）
        self.encoder_signal = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=hidden_dim//4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=hidden_dim//4, out_channels=hidden_dim//8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=hidden_dim//8, out_channels=hidden_dim//16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # 标签处理路径（MLP）
        self.label_fc = nn.Sequential(
            nn.Linear(label_dim, hidden_dim//4),
            nn.ReLU(),
            nn.Linear(hidden_dim//4, hidden_dim//8),
            nn.ReLU(),
            nn.Linear(hidden_dim//8, hidden_dim//16)
        )
        
        # 合并后的输出路径（解码器部分）
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=hidden_dim//16 + hidden_dim//16, out_channels=hidden_dim//8, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=hidden_dim//8, out_channels=hidden_dim//4, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=hidden_dim//4, out_channels=1, kernel_size=6, stride=2)
        )
        
        # 最后的输出层
        self.output_layer = nn.Linear(signal_length, output_shape)

    def forward(self, signal, label):
        # 处理信号 [batch, 1, signal_length]
        signal_features = self.encoder_signal(signal)  # 编码后的特征 [batch, hidden_dim//16, signal_length // 8]
        
        # 处理标签 [batch, label_dim]
        label_features = self.label_fc(label)  # [batch, hidden_dim//64]
        
        # 将标签特征扩展为与信号特征相同的长度
        label_features = label_features.unsqueeze(2).repeat(1, 1, signal_features.size(2))
        
        # 合并特征 [batch, hidden_dim//16 + hidden_dim//64, signal_length // 8]
        combined = torch.cat([signal_features, label_features], dim=1)

        # 解码 [batch, 1, signal_length]
        decoded = self.decoder(combined)
        
        # 输出 [batch, output_shape]
        output = self.output_layer(decoded.view(decoded.size(0), -1))
        return output.view(-1, 1, M)  # 保持输入输出形状一致

In [5]:
def load_data(generation_signal_path, experimental_signal_path, M):
    generation_signals = []  # 存储生成信号
    labels = []  # 存储标签 (Force_label, HI_label, Distance_label, Time_label)
    experimental_signals = []  # 存储输出参数
    filenames = []  # 存储文件名

    # 加载生成信号数据
    generation_dataPaths = sorted(glob.glob(os.path.join(generation_signal_path, '**', '*.csv'), recursive=True))
    for dataPath in generation_dataPaths:

        filename = os.path.basename(dataPath)
        parts = filename.split('_')
        Force_label = float(parts[0])
        HI_label = float(parts[1])
        Distance_label = float(parts[3])
        Time_label = float(parts[2])

        data_1 = pd.read_csv(dataPath)
        for N in range(len(data_1)):
            if data_1.iloc[N, 1] != 0:
                break
        if N < len(data_1):
            generation_signal = data_1.iloc[N:N+M, 1].values
        else:
            continue

        # 分别存储 generation_signal 和 labels
        generation_signals.append(generation_signal)
        labels.append([Force_label, HI_label, Distance_label, Time_label])
        filenames.append(filename)

    # 加载实验信号数据
    experimental_dataPaths = sorted(glob.glob(os.path.join(experimental_signal_path, '**', '*.csv'), recursive=True))
    for dataPath in experimental_dataPaths:
        data_1 = pd.read_csv(dataPath)
        filename = os.path.basename(dataPath)

        for N in range(len(data_1)):
            if data_1.iloc[N, 1] != 0:
                break
        if N < len(data_1):
            experimental_signal = data_1.iloc[N:N+M, 1].values
            experimental_signals.append(experimental_signal)  # 将 experimental_signal 添加到对应的 outputs 元素中

    # 确保 generation_signals、labels 和 outputs 的长度一致
    if len(generation_signals) != len(experimental_signals) or len(labels) != len(experimental_signals):
        print("Error: The number of inputs and outputs does not match.")
        return None, None, None, None, None

    # 转换为 NumPy 数组
    generation_signals = np.array(generation_signals, dtype=float)
    labels = np.array(labels, dtype=float)
    experimental_signals = np.array(experimental_signals, dtype=float)

    return generation_signals, labels, experimental_signals, filenames

In [ ]:
generation_signal_path = f'{Data_path}Hanning_signal_generation'
experimental_signal_path = f'{Data_path}Hanning_Signal_Experiment(10000)'

# 加载数据
filenames = os.listdir(generation_signal_path)  # 假设两个文件夹的文件名是一一对应的

# 控制加载的数据量
n = 100  # 假设只加载 n% 的数据
num_samples = int(len(filenames) * n / 100)
sampled_filenames = random.sample(filenames, num_samples)

# 根据采样的文件名加载数据
# 假设 load_data 函数已经定义好
generation_signals, labels, experimental_signal, filenames = load_data(generation_signal_path, experimental_signal_path, M)

# 将所有数据打乱
shuffled_indices = np.arange(len(generation_signals))
np.random.shuffle(shuffled_indices)

generation_signals = generation_signals[shuffled_indices]
labels = labels[shuffled_indices]
experimental_signal = experimental_signal[shuffled_indices]

test_filenames = [filename for filename in filenames]

generation_signals_test = generation_signals
labels_test = labels
experimental_signal_test = experimental_signal

generation_signals_test = np.expand_dims(generation_signals_test, axis=1)
experimental_signal_test = np.expand_dims(experimental_signal_test, axis=1)

print(f'generation_signals_test:{generation_signals_test.shape}')

# 创建数据集
test_dataset = SignalDataset(generation_signals_test, labels_test, experimental_signal_test)

# 创建数据加载器
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [ ]:
generation_signals_test_1 = generation_signals_test.reshape(-1,M,1)
plt.style.use('default')
plt.figure(figsize=(10,6))
plt.rcParams['font.family'] = ['Times New Roman']
plt.plot(generation_signals_test_1[1],linewidth=1.5)
plt.xlabel('Sample point',fontdict={'weight': 'normal', 'size': 18})
plt.ylabel('Amplitude(v)',fontdict={'weight': 'normal', 'size': 18})
#坐标轴刻度大小设置
plt.tick_params(axis='both', which='major', labelsize=15)
plt.xlim([0,M])
plt.savefig(f'{save_path}Forward_problem/Generation_signal_test.jpg', dpi=600, bbox_inches='tight')

In [ ]:
experimental_signal_test_1 = experimental_signal_test.reshape(-1,M,1)
plt.style.use('default')
plt.figure(figsize=(10,6))
plt.rcParams['font.family'] = ['Times New Roman']
plt.plot(experimental_signal_test_1[25],linewidth=1.5)
plt.xlabel('Sample point',fontdict={'weight': 'normal', 'size': 18})
plt.ylabel('Amplitude(V)',fontdict={'weight': 'normal', 'size': 18})
#坐标轴刻度大小设置
plt.tick_params(axis='both', which='major', labelsize=15)
plt.xlim([0,M])
plt.savefig(f'{save_path}Forward_problem/Experiment_signal_test.jpg', dpi=600, bbox_inches='tight')

In [ ]:
# 训练模型
model=torch.load(f'{save_path}Forward_problem/Forward_Model.pth')  # 加载训练好的模型参数
model.to(device)

In [10]:
def calculate_fft(signal, sampling_rate):
    N = len(signal)  # 信号长度
    fft_values = np.fft.fft(signal)  # 计算FFT
    fft_magnitude = np.abs(fft_values)  # 取模
    fft_magnitude = fft_magnitude[:N // 2]  # 只取一半（正频率部分）
    freq = np.fft.fftfreq(N, d=1 / sampling_rate)[:N // 2]  # 频率轴
    return freq, fft_magnitude

In [11]:
def calculate_pcc(x, y):
    """
    计算两个信号的皮尔逊相关系数(PCC)。

    参数:
        x (torch.Tensor): 第一个信号，假定在 CUDA 上。
        y (torch.Tensor): 第二个信号，假定在 CUDA 上。

    返回:
        float: 皮尔逊相关系数。
    """
    
    # 计算均值
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    
    # 计算协方差
    covariance = torch.mean((x - mean_x) * (y - mean_y))
    
    # 计算标准差
    std_x = torch.std(x)
    std_y = torch.std(y)
    
    # 计算皮尔逊相关系数
    if std_x == 0 or std_y == 0:
        raise ValueError("标准差不能为零，这可能导致除以零的错误。")
    
    pcc = covariance / (std_x * std_y)

    pcc_1 = pcc.to(device)

    return pcc_1.item()  # 将结果转换为 Python 标量

In [ ]:
pcc_time = 0.0
pcc_fft = 0.0

Error_fft = 0.0

# 采样率
sampling_rate = 2.5e7

signal_predicted = []
signal_origin=[]

running_loss = 0.0
criterion = nn.MSELoss().to(device)

# 将模型设置为评估模式
model.eval()  
# 进行预测
with torch.no_grad():  # 关闭梯度计算
    for batch in test_loader:
        generation_signal = batch['generation_signal'].to(device)
        labels = batch['labels'].to(device)
        experimental_signal = batch['experimental_signal'].to(device)

        experimental_signal_pred = model(generation_signal, labels)
        loss = criterion(experimental_signal_pred, experimental_signal)

        running_loss += loss.item()
        pcc_time += calculate_pcc(experimental_signal_pred,experimental_signal)

        # 将重构信号移动到CPU并转换为NumPy数组
        Origin_x3 = experimental_signal.cpu().numpy().squeeze()
        predicted_x3 = experimental_signal_pred.cpu().numpy().squeeze()

        # 假设 Original_signal 和 predicted_signal 已经定义
        freq_origin, fft_magnitude_origin = calculate_fft(Origin_x3, sampling_rate)
        freq_predicted, fft_magnitude_predicted = calculate_fft(predicted_x3, sampling_rate)

        fft_magnitude_predicted,fft_magnitude_origin=torch.from_numpy(fft_magnitude_predicted),torch.from_numpy(fft_magnitude_origin),

        Error_fft += criterion(fft_magnitude_predicted,fft_magnitude_origin)
        pcc_fft += calculate_pcc(fft_magnitude_predicted,fft_magnitude_origin)
    
        signal_origin.append(experimental_signal)
        signal_predicted.append(experimental_signal_pred)

    test_loss_time = running_loss / len(test_loader)
    test_pcc_time = pcc_time / len(test_loader)

    test_fft_error = Error_fft / len(test_loader)
    test_fft_pcc = pcc_fft / len(test_loader)

    print(f'Time_error:{test_loss_time:.4f}')
    print(f'Time_PCC:{test_pcc_time:.4f}')
    print(f'FFT_error:{test_fft_error:.4f}')
    print(f'FFT_PCC:{test_fft_pcc:.4f}')

In [ ]:
folder_path = f'{Data_path}Hanning_Signal_Experiment(10000)'  # 替换为你的文件夹路径
file_names = [file_name for file_name in os.listdir(folder_path) if file_name.endswith(".csv")]

for a in range(0, len(file_names), 1):
    file_name = file_names[a]  # 获取对应的文件名
    print(f"Processing file: {file_name}")

    # 假设 experimental_signals, signal_origin, signal_predicted 已定义
    
    # 将重构信号移动到CPU并转换为NumPy数
    signal_origin_3 = signal_origin[a].cpu().numpy().squeeze()
    predicted_signal = signal_predicted[a].cpu().numpy().squeeze()

    # 保存 predicted_signal 到 CSV 文件
    save_path = f'{Data_path}Prediction_signal_FFT_Amp'  # 保存路径
    os.makedirs(save_path, exist_ok=True)  # 如果路径不存在，则创建
    time = np.arange(len(predicted_signal)) * 4e-8  # 时间序列，起点为 0，间隔为 4e-8 秒
    data_to_save = np.column_stack((time, predicted_signal))  # 将时间和信号合并为二维数组
    np.savetxt(os.path.join(save_path, file_name), data_to_save, delimiter=',', header="Time(s),Amplitude(V)", comments='')  # 保存为 CSV 文件
    print(f"Saved predicted signal to {os.path.join(save_path, file_name)}")